In [ ]:
import pandas as pd
import numpy as np
import os

data_salareis = "raw_data/Salaries.csv"
data_batting = "raw_data/Batting.csv"
data_pitching = "raw_data/Pitching.csv"
data_halloffame = "raw_data/HallOfFame.csv"
data_fielding = "raw_data/Fielding.csv"

salary_data = pd.read_csv(data_salaries)
pitching_data = pd.read_csv(data_pitchin)
rename_these = {"G": "games", "W": "wins", "L": "losses", "H": "hits",
              "WP": "wild_pitches", "R": "runs_allowed", "SO": "strikeouts",
              "SHO": "shutouts", "SV": "saves", "IPouts": "outs_pitched",
              "BB": "walks", "BFP": "batters_faced"}
pitching_data.rename(columns=rename_these, inplace=True)